In [1]:
# -*- coding: utf-8 -*-
'''An implementation of sequence to sequence learning for performing addition

Input: "535+61"
Output: "596"
Padding is handled by using a repeated sentinel character (space)

Input may optionally be reversed, shown to increase performance in many tasks in:
"Learning to Execute"
http://arxiv.org/abs/1410.4615
and
"Sequence to Sequence Learning with Neural Networks"
http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
Theoretically it introduces shorter term dependencies between source and target.

Two digits reversed:
+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs

Three digits reversed:
+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs

Four digits reversed:
+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs

Five digits reversed:
+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs
'''  # noqa

from __future__ import print_function

#add 
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import numpy as np
from six.moves import range


In [3]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.

        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.

        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'


In [ ]:

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

# Try replacing GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last output of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


Generating data...
Total addition questions: 50000
Vectorization...
Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               72192     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 4, 128)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed (TimeDistri (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________

--------------------------------------------------
Iteration 1
Train on 45000 samples, validat

45000/45000 [==============================] - 23s 505us/step - loss: 0.6211 - acc: 0.7740 - val_loss: 0.5522 - val_acc: 0.7971
Q 12+85   T 97   ☒ 98  
Q 381+422 T 803  ☒ 808 
Q 272+229 T 501  ☒ 502 
Q 167+568 T 735  ☑ 735 
Q 962+7   T 969  ☑ 969 
Q 727+5   T 732  ☑ 732 
Q 166+79  T 245  ☒ 246 
Q 168+776 T 944  ☑ 944 
Q 157+0   T 157  ☒ 158 
Q 871+19  T 890  ☑ 890 

--------------------------------------------------
Iteration 14
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 23s 507us/step - loss: 0.4579 - acc: 0.8408 - val_loss: 0.3751 - val_acc: 0.8803
Q 879+65  T 944  ☑ 944 
Q 82+135  T 217  ☑ 217 
Q 879+737 T 1616 ☒ 1614
Q 906+1   T 907  ☒ 906 
Q 98+5    T 103  ☑ 103 
Q 56+66   T 122  ☑ 122 
Q 939+87  T 1026 ☒ 1025
Q 379+37  T 416  ☒ 415 
Q 493+549 T 1042 ☑ 1042
Q 687+84  T 771  ☑ 771 

--------------------------------------------------
Iteration 15
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

45000/45000 [==============================] - 23s 512us/step - loss: 0.0254 - acc: 0.9957 - val_loss: 0.0631 - val_acc: 0.9797
Q 700+836 T 1536 ☑ 1536
Q 67+753  T 820  ☑ 820 
Q 16+722  T 738  ☑ 738 
Q 47+62   T 109  ☑ 109 
Q 37+64   T 101  ☑ 101 
Q 34+969  T 1003 ☑ 1003
Q 466+832 T 1298 ☑ 1298
Q 73+697  T 770  ☑ 770 
Q 7+757   T 764  ☑ 764 
Q 44+261  T 305  ☑ 305 

--------------------------------------------------
Iteration 28
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 25s 552us/step - loss: 0.0291 - acc: 0.9937 - val_loss: 0.0210 - val_acc: 0.9965
Q 361+6   T 367  ☑ 367 
Q 138+74  T 212  ☑ 212 
Q 67+778  T 845  ☑ 845 
Q 12+63   T 75   ☑ 75  
Q 437+64  T 501  ☑ 501 
Q 992+358 T 1350 ☑ 1350
Q 52+616  T 668  ☑ 668 
Q 934+16  T 950  ☑ 950 
Q 933+615 T 1548 ☑ 1548
Q 52+301  T 353  ☑ 353 

--------------------------------------------------
Iteration 29
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

45000/45000 [==============================] - 24s 523us/step - loss: 0.0034 - acc: 0.9998 - val_loss: 0.0060 - val_acc: 0.9990
Q 14+230  T 244  ☑ 244 
Q 46+994  T 1040 ☑ 1040
Q 50+628  T 678  ☑ 678 
Q 483+21  T 504  ☑ 504 
Q 678+80  T 758  ☑ 758 
Q 476+1   T 477  ☑ 477 
Q 44+297  T 341  ☑ 341 
Q 983+476 T 1459 ☑ 1459
Q 49+283  T 332  ☑ 332 
Q 400+84  T 484  ☑ 484 

--------------------------------------------------
Iteration 42
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 24s 523us/step - loss: 0.0376 - acc: 0.9882 - val_loss: 0.0679 - val_acc: 0.9761
Q 6+210   T 216  ☑ 216 
Q 96+244  T 340  ☑ 340 
Q 36+39   T 75   ☑ 75  
Q 50+822  T 872  ☑ 872 
Q 3+76    T 79   ☒ 89  
Q 74+669  T 743  ☑ 743 
Q 694+90  T 784  ☑ 784 
Q 51+480  T 531  ☑ 531 
Q 33+939  T 972  ☑ 972 
Q 32+608  T 640  ☑ 640 

--------------------------------------------------
Iteration 43
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

45000/45000 [==============================] - 25s 554us/step - loss: 0.0033 - acc: 0.9998 - val_loss: 0.0049 - val_acc: 0.9991
Q 644+44  T 688  ☑ 688 
Q 47+153  T 200  ☑ 200 
Q 52+621  T 673  ☑ 673 
Q 67+320  T 387  ☑ 387 
Q 47+242  T 289  ☑ 289 
Q 1+57    T 58   ☑ 58  
Q 768+48  T 816  ☑ 816 
Q 125+188 T 313  ☑ 313 
Q 335+266 T 601  ☑ 601 
Q 687+97  T 784  ☑ 784 

--------------------------------------------------
Iteration 56
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 26s 582us/step - loss: 0.0019 - acc: 1.0000 - val_loss: 0.0040 - val_acc: 0.9993
Q 6+264   T 270  ☑ 270 
Q 245+82  T 327  ☑ 327 
Q 275+64  T 339  ☑ 339 
Q 369+60  T 429  ☑ 429 
Q 299+71  T 370  ☑ 370 
Q 35+509  T 544  ☑ 544 
Q 11+81   T 92   ☑ 92  
Q 212+98  T 310  ☑ 310 
Q 11+852  T 863  ☑ 863 
Q 96+900  T 996  ☑ 996 

--------------------------------------------------
Iteration 57
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===